# ส่วนที่ 2: การแปลงข้อมูล (Data Transformation)

Notebook นี้แสดงขั้นตอนการแปลงข้อมูลให้พร้อมสำหรับการวิเคราะห์และจัดเก็บในฐานข้อมูล

## 1. ติดตั้งแพ็คเกจที่จำเป็น

In [ ]:
# ติดตั้งแพ็คเกจที่จำเป็น
!pip install pandas numpy matplotlib seaborn sqlalchemy

## 2. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# ตั้งค่าการแสดงผลกราฟให้สวยงาม
plt.style.use('ggplot')
sns.set(style='whitegrid')

# ตั้งค่าให้แสดงข้อมูลภาษาไทยได้
plt.rcParams['font.family'] = 'DejaVu Sans'

# แสดงเวอร์ชันของแพ็คเกจ
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")

## 3. โหลดข้อมูลจากไฟล์ที่บันทึกไว้

In [ ]:
# กำหนดพาธหลักสำหรับโปรเจค
project_path = './Employee_Data_Analytics_Project'  # ปรับตามโครงสร้างของคุณ

# แสดงไฟล์ในโปรเจค
print("ไฟล์ในโปรเจค:")
for file in os.listdir(project_path):
    print(f"- {file}")

In [ ]:
# โหลดข้อมูลจากไฟล์ที่บันทึกไว้
raw_data_path = f"{project_path}/raw_employee_data.csv"

if os.path.exists(raw_data_path):
    df = pd.read_csv(raw_data_path)
    print(f"โหลดข้อมูลจาก {raw_data_path} สำเร็จ")
    print(f"ข้อมูลมีทั้งหมด {df.shape[0]} แถว และ {df.shape[1]} คอลัมน์")
else:
    print(f"ไม่พบไฟล์ {raw_data_path}")
    print("จะโหลดข้อมูลจากไฟล์ dataset_employee.csv แทน")
    df = pd.read_csv(f"{project_path}/dataset_employee.csv")
    print(f"โหลดข้อมูลจาก dataset_employee.csv สำเร็จ")
    print(f"ข้อมูลมีทั้งหมด {df.shape[0]} แถว และ {df.shape[1]} คอลัมน์")

# แสดงข้อมูลตัวอย่าง
print("\nตัวอย่างข้อมูล 5 แถวแรก:")
display(df.head())

## 5. การแปลงข้อมูล (Data Transformation)

### 5.1 แปลงปีจาก พ.ศ. เป็น ค.ศ. และสร้างคอลัมน์ใหม่

In [ ]:
# สร้าง DataFrame ใหม่เพื่อไม่ให้กระทบข้อมูลเดิม
df_transformed = df.copy()

# แปลงปีจาก พ.ศ. เป็น ค.ศ. และสร้างคอลัมน์ใหม่
df_transformed['YEAR_CE'] = df_transformed['YEAR'] - 543

print("ตัวอย่างข้อมูลหลังการแปลงปี:")
display(df_transformed[['YEAR', 'YEAR_CE', 'QUARTER', 'REGION', 'PROVINCE']].head())

### 5.2 แปลงไตรมาสเป็นตัวเลข (เพื่อให้เรียงลำดับได้)

In [ ]:
# แปลงไตรมาสเป็นตัวเลข
quarter_map = {
    'ไตรมาสที่ 1': 1,
    'ไตรมาสที่ 2': 2,
    'ไตรมาสที่ 3': 3,
    'ไตรมาสที่ 4': 4
}
df_transformed['QUARTER_NUM'] = df_transformed['QUARTER'].map(quarter_map)

print("ตัวอย่างข้อมูลหลังการแปลงไตรมาส:")
display(df_transformed[['QUARTER', 'QUARTER_NUM', 'YEAR', 'YEAR_CE']].head())

### 5.3 สร้างคอลัมน์วันที่โดยประมาณสำหรับช่วงกลางของแต่ละไตรมาส

In [ ]:
# สร้างคอลัมน์วันที่โดยประมาณสำหรับช่วงกลางของแต่ละไตรมาส
def get_quarter_date(row):
    year = row['YEAR_CE']
    quarter = row['QUARTER_NUM']
    if quarter == 1:
        return f"{year}-02-15"
    elif quarter == 2:
        return f"{year}-05-15"
    elif quarter == 3:
        return f"{year}-08-15"
    elif quarter == 4:
        return f"{year}-11-15"

df_transformed['ESTIMATED_DATE'] = df_transformed.apply(get_quarter_date, axis=1)
df_transformed['ESTIMATED_DATE'] = pd.to_datetime(df_transformed['ESTIMATED_DATE'])

print("ตัวอย่างข้อมูลหลังการสร้างวันที่โดยประมาณ:")
display(df_transformed[['YEAR', 'QUARTER', 'QUARTER_NUM', 'ESTIMATED_DATE']].head())

### 5.4 สร้างคอลัมน์เพิ่มเติมสำหรับการวิเคราะห์ - สัดส่วนการจ้างงานต่อภูมิภาค

In [ ]:
# คำนวณสัดส่วนการจ้างงานต่อภูมิภาค
region_totals = df_transformed.groupby(['YEAR', 'QUARTER', 'REGION'])['VALUE'].sum().reset_index()
region_totals.rename(columns={'VALUE': 'REGION_TOTAL'}, inplace=True)

df_transformed = pd.merge(df_transformed, region_totals, on=['YEAR', 'QUARTER', 'REGION'], how='left')
df_transformed['PERCENT_OF_REGION'] = (df_transformed['VALUE'] / df_transformed['REGION_TOTAL'] * 100).round(2)

print("ตัวอย่างข้อมูลหลังการคำนวณสัดส่วนต่อภูมิภาค:")
display(df_transformed[['REGION', 'PROVINCE', 'VALUE', 'REGION_TOTAL', 'PERCENT_OF_REGION']].head(10))

### 5.5 คำนวณอัตราการเปลี่ยนแปลงตามไตรมาส (QoQ)

In [ ]:
# คำนวณอัตราการเปลี่ยนแปลงตามไตรมาส (QoQ)
df_sorted = df_transformed.sort_values(['PROVINCE', 'SEX', 'YEAR', 'QUARTER_NUM'])
df_transformed['PREV_VALUE'] = df_sorted.groupby(['PROVINCE', 'SEX'])['VALUE'].shift(1)
df_transformed['QOQ_CHANGE_PCT'] = ((df_transformed['VALUE'] - df_transformed['PREV_VALUE']) / df_transformed['PREV_VALUE'] * 100).round(2)

print("ตัวอย่างข้อมูลหลังการคำนวณอัตราการเปลี่ยนแปลง QoQ:")
sample_province = df_transformed[df_transformed['PROVINCE'] == df_transformed['PROVINCE'].iloc[0]]
sample_province = sample_province.sort_values(['YEAR', 'QUARTER_NUM'])
display(sample_province[['PROVINCE', 'YEAR', 'QUARTER', 'VALUE', 'PREV_VALUE', 'QOQ_CHANGE_PCT']].head(8))

### 5.6 คำนวณอัตราการเปลี่ยนแปลงเทียบกับปีก่อน (YoY)

In [ ]:
# คำนวณอัตราการเปลี่ยนแปลงเทียบกับปีก่อน (YoY)
df_transformed['YOY_PREV_VALUE'] = df_sorted.groupby(['PROVINCE', 'SEX', 'QUARTER_NUM'])['VALUE'].shift(4)  # ย้อนหลัง 4 ไตรมาส
df_transformed['YOY_CHANGE_PCT'] = ((df_transformed['VALUE'] - df_transformed['YOY_PREV_VALUE']) / df_transformed['YOY_PREV_VALUE'] * 100).round(2)

print("ตัวอย่างข้อมูลหลังการคำนวณอัตราการเปลี่ยนแปลง YoY:")
sample_province = df_transformed[df_transformed['PROVINCE'] == df_transformed['PROVINCE'].iloc[0]]
sample_province = sample_province.sort_values(['YEAR', 'QUARTER_NUM'])
display(sample_province[['PROVINCE', 'YEAR', 'QUARTER', 'VALUE', 'YOY_PREV_VALUE', 'YOY_CHANGE_PCT']].head(8))

### 5.7 คำนวณค่าเฉลี่ยเคลื่อนที่ 4 ไตรมาส

In [ ]:
# คำนวณค่าเฉลี่ยเคลื่อนที่ 4 ไตรมาส
df_transformed['ROLLING_AVG_4Q'] = df_sorted.groupby(['PROVINCE', 'SEX'])['VALUE'].transform(lambda x: x.rolling(window=4, min_periods=1).mean()).round(2)

print("ตัวอย่างข้อมูลหลังการคำนวณค่าเฉลี่ยเคลื่อนที่:")
sample_province = df_transformed[df_transformed['PROVINCE'] == df_transformed['PROVINCE'].iloc[0]]
sample_province = sample_province.sort_values(['YEAR', 'QUARTER_NUM'])
display(sample_province[['PROVINCE', 'YEAR', 'QUARTER', 'VALUE', 'ROLLING_AVG_4Q']].head(8))

### 5.8 เพิ่มคอลัมน์รายวันที่สำหรับการจัดกลุ่ม (เพื่อให้วิเคราะห์ได้หลากหลายมิติ)

In [ ]:
# เพิ่มคอลัมน์รายวันที่สำหรับการจัดกลุ่ม
df_transformed['YEAR_QUARTER'] = df_transformed['YEAR'].astype(str) + '-Q' + df_transformed['QUARTER_NUM'].astype(str)
df_transformed['YEAR_QUARTER_CE'] = df_transformed['YEAR_CE'].astype(str) + '-Q' + df_transformed['QUARTER_NUM'].astype(str)

print("ตัวอย่างข้อมูลหลังการเพิ่มคอลัมน์รายวันที่:")
display(df_transformed[['YEAR', 'QUARTER', 'YEAR_QUARTER', 'YEAR_CE', 'YEAR_QUARTER_CE']].head())

## 6. ตรวจสอบข้อมูลที่แปลงแล้ว

In [ ]:
# แสดงโครงสร้างข้อมูลหลังการแปลง
print("โครงสร้างข้อมูลหลังการแปลง:")
print(f"จำนวนแถว: {df_transformed.shape[0]}")
print(f"จำนวนคอลัมน์: {df_transformed.shape[1]}")

# แสดงรายชื่อคอลัมน์ทั้งหมด
print("\nรายชื่อคอลัมน์ทั้งหมด:")
print(df_transformed.columns.tolist())

# แสดงตัวอย่างข้อมูลที่แปลงแล้ว
print("\nตัวอย่างข้อมูลที่แปลงแล้ว:")
display(df_transformed.head())

# ตรวจสอบข้อมูลที่ขาดหาย
print("\nตรวจสอบข้อมูลที่ขาดหาย:")
missing_data = df_transformed.isnull().sum()
missing_data = missing_data[missing_data > 0]  # แสดงเฉพาะคอลัมน์ที่มีข้อมูลขาดหาย
if len(missing_data) > 0:
    display(missing_data)
else:
    print("ไม่มีข้อมูลที่ขาดหาย")

## 7. จัดการข้อมูลที่ขาดหาย (ถ้ามี)

In [ ]:
# จัดการข้อมูลที่ขาดหาย (ถ้ามี)
# เนื่องจากมีค่า PREV_VALUE, QOQ_CHANGE_PCT, YOY_PREV_VALUE, YOY_CHANGE_PCT ที่ขาดหาย (ไตรมาสแรกไม่มีค่าก่อนหน้า)
# เราจะไม่เติมค่าเหล่านี้ แต่จะให้เป็น NaN เพื่อไม่ให้บิดเบือนข้อมูล

# แสดงจำนวนแถวที่มีการขาดหายในแต่ละคอลัมน์
missing_counts = df_transformed.isnull().sum()
print("จำนวนแถวที่มีการขาดหายในแต่ละคอลัมน์:")
missing_counts = missing_counts[missing_counts > 0]
if len(missing_counts) > 0:
    display(missing_counts)
    
    # ตรวจสอบว่าการขาดหายข้อมูลเกิดขึ้นที่ไตรมาสแรกหรือไม่
    first_quarters = df_transformed[df_transformed['PREV_VALUE'].isnull()][['PROVINCE', 'SEX', 'YEAR', 'QUARTER']].head()
    print("\nตัวอย่างข้อมูลที่ขาดหาย:")
    display(first_quarters)
else:
    print("ไม่มีข้อมูลที่ขาดหาย")

## 8. สร้างตารางสำหรับการวิเคราะห์เพิ่มเติม

In [ ]:
# 1. ตารางสรุปตามภูมิภาค
region_summary = df_transformed.groupby(['YEAR', 'QUARTER', 'REGION', 'SEX']).agg(
    total_employees=('VALUE', 'sum'),
    avg_employees=('VALUE', 'mean'),
    min_employees=('VALUE', 'min'),
    max_employees=('VALUE', 'max'),
    province_count=('PROVINCE', 'nunique')
).reset_index()

print("ตารางสรุปตามภูมิภาค:")
display(region_summary.head())

# 2. ตารางสรุปตามปีและไตรมาส
time_summary = df_transformed.groupby(['YEAR', 'QUARTER', 'YEAR_QUARTER']).agg(
    total_employees=('VALUE', 'sum')
).reset_index()

# เพิ่มข้อมูลแยกตามเพศ
male_data = df_transformed[df_transformed['SEX'] == 'ชาย'].groupby(['YEAR', 'QUARTER', 'YEAR_QUARTER'])['VALUE'].sum().reset_index()
male_data.rename(columns={'VALUE': 'male_employees'}, inplace=True)

female_data = df_transformed[df_transformed['SEX'] == 'หญิง'].groupby(['YEAR', 'QUARTER', 'YEAR_QUARTER'])['VALUE'].sum().reset_index()
female_data.rename(columns={'VALUE': 'female_employees'}, inplace=True)

# รวมข้อมูล
time_summary = pd.merge(time_summary, male_data, on=['YEAR', 'QUARTER', 'YEAR_QUARTER'], how='left')
time_summary = pd.merge(time_summary, female_data, on=['YEAR', 'QUARTER', 'YEAR_QUARTER'], how='left')

# คำนวณสัดส่วนเพศ
time_summary['male_ratio'] = (time_summary['male_employees'] / time_summary['total_employees'] * 100).round(2)
time_summary['female_ratio'] = (time_summary['female_employees'] / time_summary['total_employees'] * 100).round(2)

print("\nตารางสรุปตามปีและไตรมาส:")
display(time_summary.head())

# 3. ตารางสรุปตามจังหวัด
province_summary = df_transformed.groupby(['PROVINCE', 'SEX']).agg(
    avg_employees=('VALUE', 'mean'),
    min_employees=('VALUE', 'min'),
    max_employees=('VALUE', 'max'),
    count=('VALUE', 'count')
).reset_index()

print("\nตารางสรุปตามจังหวัด:")
display(province_summary.head())

## 9. บันทึกข้อมูลที่แปลงแล้ว

In [ ]:
# บันทึกข้อมูลที่แปลงแล้ว
transformed_data_path = f"{project_path}/transformed_employee_data.csv"
df_transformed.to_csv(transformed_data_path, index=False)
print(f"บันทึกข้อมูลที่แปลงแล้วไปยัง {transformed_data_path} สำเร็จ")

# บันทึกตารางสรุป
region_summary_path = f"{project_path}/region_summary.csv"
region_summary.to_csv(region_summary_path, index=False)
print(f"บันทึกตารางสรุปตามภูมิภาคไปยัง {region_summary_path} สำเร็จ")

time_summary_path = f"{project_path}/time_summary.csv"
time_summary.to_csv(time_summary_path, index=False)
print(f"บันทึกตารางสรุปตามปีและไตรมาสไปยัง {time_summary_path} สำเร็จ")

province_summary_path = f"{project_path}/province_summary.csv"
province_summary.to_csv(province_summary_path, index=False)
print(f"บันทึกตารางสรุปตามจังหวัดไปยัง {province_summary_path} สำเร็จ")

## 10. สรุปผลการแปลงข้อมูล

In [ ]:
# สรุปผลการแปลงข้อมูล
print("สรุปผลการแปลงข้อมูล:")
print(f"1. แปลงข้อมูลจำนวน {df.shape[0]} แถว")
print(f"2. เพิ่มคอลัมน์ใหม่จาก {df.shape[1]} คอลัมน์ เป็น {df_transformed.shape[1]} คอลัมน์")
print("3. เพิ่มคอลัมน์ใหม่:")
new_columns = set(df_transformed.columns) - set(df.columns)
for col in new_columns:
    print(f"   - {col}")
print("4. สร้างตารางสรุปเพิ่มเติม:")
print(f"   - ตารางสรุปตามภูมิภาค ({region_summary.shape[0]} แถว, {region_summary.shape[1]} คอลัมน์)")
print(f"   - ตารางสรุปตามปีและไตรมาส ({time_summary.shape[0]} แถว, {time_summary.shape[1]} คอลัมน์)")
print(f"   - ตารางสรุปตามจังหวัด ({province_summary.shape[0]} แถว, {province_summary.shape[1]} คอลัมน์)")

## 11. ข้อมูลเสร็จพร้อมสำหรับการโหลดลงฐานข้อมูล (Data Loading)

ดำเนินการแปลงข้อมูลเสร็จสิ้น ข้อมูลพร้อมสำหรับขั้นตอนการโหลดข้อมูลลงฐานข้อมูล SQLite ในไฟล์ `03_Data_Loading_SQLite.ipynb`